# Data Preprocessing 

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from fastai.imports import *
from fastai.tabular.all import *

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import eli5
from eli5.sklearn import PermutationImportance


from scipy.stats import skew, norm
from catboost import CatBoostRegressor
from feature_engine.transformation import YeoJohnsonTransformer
from category_encoders.binary import BinaryEncoder

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import shap
import xgboost
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from mlxtend.regressor import StackingCVRegressor

In [3]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

In [4]:
train.shape, test.shape

((1460, 81), (1459, 80))

Based on an initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for the data preprocessing work. 

In [5]:
train_test = pd.concat([train, test], ignore_index=True)

In [6]:
train_test.shape

(2919, 81)

To show the amount of 'NaN' values in each column of the dataset. 

In [7]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. At first those columns were eliminated. However, the results were better if these columns were retained and the missing data were coded as 'None', assuming these features didn't exist for their related houses. 

There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

In [8]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test[drop_high_nan] = train_test[drop_high_nan].fillna('None')

small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable.

In [9]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)

garage_cols = ['GarageType', 'GarageFinish', 'GarageQual','GarageCond', 'GarageYrBlt']
train_test[garage_cols] = train_test[garage_cols].fillna('None')

train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)

A final check to ensure that the only missing values are the expected SalePrice data from the test dataset. 

In [10]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

SalePrice    1459
dtype: int64

The BsmtQual and BsmtFinType1 have ordinal scaled categorical values with an inherent order. So the conversion from categorical to numerical was done manually. 

In [11]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [12]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'LwQ'], dtype=object)

In [13]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

To look at how many features have a skew above 0.6, since high skew can be an issue in regression analysis. For some reason, the results were better with a skew limit of 0.6 than the more standard value of 0.5. Skewness measures how symmetrical a distribution of data is. Data with a skew of 0 is perfectly symmetrical.

In [14]:
number_cols = train_test.select_dtypes(include=np.number).columns.tolist()

In [15]:
skew_features = train_test[number_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.6]
skew_index = high_skew.index
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', '3SsnPorch',
       'KitchenAbvGr', 'EnclosedPorch', 'ScreenPorch', 'OpenPorchSF',
       'WoodDeckSF', 'LotFrontage', '1stFlrSF', 'MSSubClass', 'GrLivArea',
       '2ndFlrSF', 'BsmtQual', 'TotRmsAbvGrd', 'Fireplaces', 'HalfBath'],
      dtype='object')

In [16]:
print("There are {} numerical features with Skew > 0.6 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skewness

There are 19 numerical features with Skew > 0.6 :


,Skew
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
KitchenAbvGr,4.302254
EnclosedPorch,4.003891
ScreenPorch,3.946694
OpenPorchSF,2.535114
WoodDeckSF,1.842433


To normalize the features with skew above 0.6 the Yeo-Johnson Transformation is used since data with negative values or values of zero can be included. This transformation is a way to transform a continuous variable so that the output looks more normally distributed. 

In [17]:
yjt = YeoJohnsonTransformer()
yjt.fit(train_test[skew_index])
train_test[skew_index] = yjt.transform(train_test[skew_index])

Since most models cannot work with categorical data, it will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. The best approach for this conversion seems to be the straightforward python factorize function. 

In [18]:
obj_cols = list(train_test.select_dtypes(['object']).columns)

In [19]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

To create some new columns that might compound the effects of the existing columns with higher impact on using the xgbregressor model. This work was done on an earlier workbook version. The get_score function with 'gain' as the importance type was used. This function takes the average gain of the tree splits. 

In [20]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

To create a column with the log of the SalePrice to match the evaluation metric in the contest. 

In [21]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

#  Setting Up and Running the Models

To separate the train_test dataset back into the train and test datasets and identify the independent and dependent columns. Because the dataset is so small, the cross fold validation process seemed to have much less overfitting than creating the separate validation and training sets. 

In [22]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [23]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

To set up the cross validation folds.

In [24]:
kf = KFold(n_splits=12, random_state=42, shuffle=True)

In a separate workbook, Optuna was used to find the optimal parameters for a wide selection of regression models. It was interesting that the results using Optuna were not significantly better than using the default parameters for most of the models. 

In [25]:
cat_model = CatBoostRegressor(colsample_bylevel= 0.08309602563537534, learning_rate= 0.08286145675756133, depth= 4, 
                              l2_leaf_reg= 14.555249413444315, subsample= 0.9097411584295835, 
                              bagging_temperature= 3.177590955252409, model_size_reg= 0.3808343022980778, 
                              boosting_type= 'Plain')

xgb_model = xgb.XGBRegressor(reg_lambda= 5.096430599223015, alpha= 1.3223698329753615, 
                             colsample_bytree= 0.10366646849522436, subsample= 0.9705075073357825, 
                             learning_rate= 0.04681670801808327, n_estimators= 2340, max_depth= 3, 
                             min_child_weight= 2, num_parallel_tree= 1,
                             objective= 'reg:linear', random_state=42)

lr_model = LinearRegression()

br_model = BayesianRidge(alpha_init= 1.4228919418021304, lambda_init= 0.03899454131279402, compute_score=True)

lightgbm_model = LGBMRegressor(objective='regression', 
                       num_leaves=9,
                       learning_rate=0.0019200325083996161, 
                       n_estimators=7000,
                       max_bin=200, 
                       subsample= 0.40495582495561083, 
                       bagging_fraction=0.4240987934393466,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.1309662269268009,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 1,
                       verbose=-1,
                       random_state=42)

svr_model = make_pipeline(RobustScaler(), SVR(C=29.994348636600485, epsilon= 0.019482341995527, 
                                              gamma=0.0009992835981754227))

dtr_model = DecisionTreeRegressor()

rfr_model = RandomForestRegressor(n_estimators=1200,
                          max_depth=10,
                          min_weight_fraction_leaf=0,
                          min_samples_split=5,
                          min_samples_leaf=1,
                          max_leaf_nodes=90,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

gbr_model = GradientBoostingRegressor(n_estimators=2200,
                                learning_rate=0.05005099614548874,
                                max_depth=2,
                                max_features='sqrt',
                                min_samples_leaf=3,
                                min_samples_split=24,
                                loss='huber',
                                random_state=42)

ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 
                18, 20, 30, 50, 75, 100]
ridge_model = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

To define the scoring metric. 

In [26]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return rmse

To define the model names and models and then create a loop to run each model and print out their scores. 

In [27]:
model_names = ['CatBoostRegressor','XGBRegressor','LinearRegression','BayesianRidge','LGBMRegressor',
               'SVR','DecisionTreeRegressor','RandomForestRegressor','GradientBoostingRegressor','RidgeRegressor']

In [28]:
models = [cat_model,xgb_model,lr_model,br_model,lightgbm_model,svr_model,dtr_model,rfr_model,gbr_model,ridge_model]

In [29]:
score_rmse = []
for i in models:
    rmse = np.sqrt(-cross_val_score(i, X, y, scoring="neg_mean_squared_error", cv=kf))
    ave_rmse = np.mean(rmse)
    score_rmse.append(ave_rmse)

0:	learn: 0.3844132	total: 139ms	remaining: 2m 18s
1:	learn: 0.3675341	total: 140ms	remaining: 1m 10s
2:	learn: 0.3525793	total: 142ms	remaining: 47.1s
3:	learn: 0.3386058	total: 143ms	remaining: 35.5s
4:	learn: 0.3275463	total: 144ms	remaining: 28.6s
5:	learn: 0.3147258	total: 145ms	remaining: 24s
6:	learn: 0.3052153	total: 146ms	remaining: 20.7s
7:	learn: 0.2947593	total: 147ms	remaining: 18.3s
8:	learn: 0.2836075	total: 148ms	remaining: 16.3s
9:	learn: 0.2738717	total: 150ms	remaining: 14.8s
10:	learn: 0.2656770	total: 151ms	remaining: 13.6s
11:	learn: 0.2574640	total: 152ms	remaining: 12.5s
12:	learn: 0.2500886	total: 153ms	remaining: 11.6s
13:	learn: 0.2428996	total: 154ms	remaining: 10.9s
14:	learn: 0.2360489	total: 155ms	remaining: 10.2s
15:	learn: 0.2297101	total: 157ms	remaining: 9.63s
16:	learn: 0.2256423	total: 158ms	remaining: 9.12s
17:	learn: 0.2204713	total: 159ms	remaining: 8.68s
18:	learn: 0.2158843	total: 160ms	remaining: 8.28s
19:	learn: 0.2115626	total: 161ms	remaini

176:	learn: 0.1107047	total: 334ms	remaining: 1.55s
177:	learn: 0.1106375	total: 335ms	remaining: 1.54s
178:	learn: 0.1105841	total: 336ms	remaining: 1.54s
179:	learn: 0.1103979	total: 337ms	remaining: 1.53s
180:	learn: 0.1103214	total: 338ms	remaining: 1.53s
181:	learn: 0.1103051	total: 339ms	remaining: 1.52s
182:	learn: 0.1102087	total: 340ms	remaining: 1.52s
183:	learn: 0.1100213	total: 341ms	remaining: 1.51s
184:	learn: 0.1099309	total: 342ms	remaining: 1.51s
185:	learn: 0.1096876	total: 343ms	remaining: 1.5s
186:	learn: 0.1096614	total: 344ms	remaining: 1.5s
187:	learn: 0.1096100	total: 346ms	remaining: 1.49s
188:	learn: 0.1094205	total: 347ms	remaining: 1.49s
189:	learn: 0.1092712	total: 348ms	remaining: 1.48s
190:	learn: 0.1090777	total: 349ms	remaining: 1.48s
191:	learn: 0.1089382	total: 350ms	remaining: 1.47s
192:	learn: 0.1088492	total: 351ms	remaining: 1.47s
193:	learn: 0.1087998	total: 352ms	remaining: 1.46s
194:	learn: 0.1087126	total: 353ms	remaining: 1.46s
195:	learn: 0.

347:	learn: 0.0942088	total: 521ms	remaining: 977ms
348:	learn: 0.0941890	total: 522ms	remaining: 974ms
349:	learn: 0.0941824	total: 523ms	remaining: 972ms
350:	learn: 0.0941685	total: 524ms	remaining: 970ms
351:	learn: 0.0939562	total: 526ms	remaining: 968ms
352:	learn: 0.0938056	total: 527ms	remaining: 965ms
353:	learn: 0.0937071	total: 528ms	remaining: 963ms
354:	learn: 0.0936657	total: 529ms	remaining: 960ms
355:	learn: 0.0935730	total: 530ms	remaining: 958ms
356:	learn: 0.0934691	total: 531ms	remaining: 956ms
357:	learn: 0.0934172	total: 532ms	remaining: 953ms
358:	learn: 0.0933508	total: 533ms	remaining: 951ms
359:	learn: 0.0933237	total: 534ms	remaining: 949ms
360:	learn: 0.0933021	total: 535ms	remaining: 946ms
361:	learn: 0.0932866	total: 536ms	remaining: 944ms
362:	learn: 0.0932016	total: 537ms	remaining: 943ms
363:	learn: 0.0931563	total: 538ms	remaining: 940ms
364:	learn: 0.0930199	total: 539ms	remaining: 938ms
365:	learn: 0.0928418	total: 541ms	remaining: 936ms
366:	learn: 

523:	learn: 0.0827826	total: 705ms	remaining: 641ms
524:	learn: 0.0827653	total: 706ms	remaining: 639ms
525:	learn: 0.0826969	total: 707ms	remaining: 637ms
526:	learn: 0.0826414	total: 709ms	remaining: 636ms
527:	learn: 0.0825301	total: 710ms	remaining: 635ms
528:	learn: 0.0825247	total: 711ms	remaining: 633ms
529:	learn: 0.0825056	total: 712ms	remaining: 631ms
530:	learn: 0.0824670	total: 713ms	remaining: 629ms
531:	learn: 0.0823750	total: 714ms	remaining: 628ms
532:	learn: 0.0823695	total: 715ms	remaining: 626ms
533:	learn: 0.0823378	total: 716ms	remaining: 625ms
534:	learn: 0.0823282	total: 717ms	remaining: 623ms
535:	learn: 0.0823246	total: 718ms	remaining: 621ms
536:	learn: 0.0823223	total: 718ms	remaining: 619ms
537:	learn: 0.0823153	total: 720ms	remaining: 618ms
538:	learn: 0.0822419	total: 721ms	remaining: 616ms
539:	learn: 0.0822358	total: 722ms	remaining: 615ms
540:	learn: 0.0822150	total: 723ms	remaining: 613ms
541:	learn: 0.0821315	total: 724ms	remaining: 611ms
542:	learn: 

703:	learn: 0.0742842	total: 892ms	remaining: 375ms
704:	learn: 0.0742313	total: 893ms	remaining: 374ms
705:	learn: 0.0741562	total: 895ms	remaining: 373ms
706:	learn: 0.0741402	total: 896ms	remaining: 371ms
707:	learn: 0.0741108	total: 897ms	remaining: 370ms
708:	learn: 0.0740515	total: 899ms	remaining: 369ms
709:	learn: 0.0740312	total: 900ms	remaining: 368ms
710:	learn: 0.0739901	total: 901ms	remaining: 366ms
711:	learn: 0.0739792	total: 902ms	remaining: 365ms
712:	learn: 0.0739711	total: 903ms	remaining: 364ms
713:	learn: 0.0739339	total: 904ms	remaining: 362ms
714:	learn: 0.0739022	total: 905ms	remaining: 361ms
715:	learn: 0.0738950	total: 906ms	remaining: 359ms
716:	learn: 0.0738239	total: 907ms	remaining: 358ms
717:	learn: 0.0738171	total: 909ms	remaining: 357ms
718:	learn: 0.0737718	total: 910ms	remaining: 356ms
719:	learn: 0.0737656	total: 919ms	remaining: 358ms
720:	learn: 0.0736739	total: 921ms	remaining: 356ms
721:	learn: 0.0736694	total: 923ms	remaining: 355ms
722:	learn: 

...

739:	learn: 0.0745603	total: 1.01s	remaining: 356ms
740:	learn: 0.0745562	total: 1.01s	remaining: 355ms
741:	learn: 0.0745395	total: 1.01s	remaining: 353ms
742:	learn: 0.0745352	total: 1.02s	remaining: 352ms
743:	learn: 0.0745113	total: 1.02s	remaining: 350ms
744:	learn: 0.0744249	total: 1.02s	remaining: 349ms
745:	learn: 0.0744070	total: 1.02s	remaining: 347ms
746:	learn: 0.0743426	total: 1.02s	remaining: 346ms
747:	learn: 0.0743359	total: 1.02s	remaining: 344ms
748:	learn: 0.0743174	total: 1.02s	remaining: 343ms
749:	learn: 0.0743123	total: 1.02s	remaining: 342ms
750:	learn: 0.0742697	total: 1.03s	remaining: 340ms
751:	learn: 0.0742178	total: 1.03s	remaining: 339ms
752:	learn: 0.0741593	total: 1.03s	remaining: 337ms
753:	learn: 0.0741537	total: 1.03s	remaining: 336ms
754:	learn: 0.0740564	total: 1.03s	remaining: 334ms
755:	learn: 0.0740501	total: 1.03s	remaining: 333ms
756:	learn: 0.0740362	total: 1.03s	remaining: 332ms
757:	learn: 0.0740340	total: 1.03s	remaining: 330ms
758:	learn: 

901:	learn: 0.0689150	total: 1.2s	remaining: 130ms
902:	learn: 0.0688699	total: 1.2s	remaining: 129ms
903:	learn: 0.0688460	total: 1.2s	remaining: 128ms
904:	learn: 0.0687723	total: 1.2s	remaining: 126ms
905:	learn: 0.0687419	total: 1.21s	remaining: 125ms
906:	learn: 0.0687042	total: 1.21s	remaining: 124ms
907:	learn: 0.0687021	total: 1.21s	remaining: 122ms
908:	learn: 0.0686919	total: 1.21s	remaining: 121ms
909:	learn: 0.0686524	total: 1.21s	remaining: 120ms
910:	learn: 0.0685803	total: 1.21s	remaining: 118ms
911:	learn: 0.0685783	total: 1.21s	remaining: 117ms
912:	learn: 0.0685282	total: 1.21s	remaining: 116ms
913:	learn: 0.0684648	total: 1.22s	remaining: 114ms
914:	learn: 0.0684442	total: 1.22s	remaining: 113ms
915:	learn: 0.0684411	total: 1.22s	remaining: 112ms
916:	learn: 0.0683893	total: 1.22s	remaining: 110ms
917:	learn: 0.0683807	total: 1.22s	remaining: 109ms
918:	learn: 0.0683725	total: 1.22s	remaining: 108ms
919:	learn: 0.0683077	total: 1.22s	remaining: 106ms
920:	learn: 0.06

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

Ill-conditioned matrix (rcond=2.74251e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.75574e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.83786e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78503e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.72177e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.92576e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.70233e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.67385e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78952e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.88052e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.79337e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.82863e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.85086e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.96328e-17): result may not be ac

Ill-conditioned matrix (rcond=2.84087e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.75574e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.96231e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.67598e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.77607e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78108e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.7712e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78589e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.75491e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.75902e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.61353e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.94666e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.63092e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.86984e-17): result may not be acc

In [30]:
model_scores = list(zip(model_names, score_rmse))

In [31]:
model_scores

[('CatBoostRegressor', 0.12377861858319061),
 ('XGBRegressor', 0.12039076843055063),
 ('LinearRegression', 0.13634490032529228),
 ('BayesianRidge', 0.1329249668215079),
 ('LGBMRegressor', 0.12229608572161717),
 ('SVR', 0.13458785598565648),
 ('DecisionTreeRegressor', 0.20409527435118138),
 ('RandomForestRegressor', 0.14156761258584682),
 ('GradientBoostingRegressor', 0.12269815792691763),
 ('RidgeRegressor', 0.13144439888498063)]

To stack up all the models above except CatBoost, optimized using linear regressor. For some reason CatBoost wasn't able to run correctly using the StackingCVRegressor model. The decision tree regressor model was also removed since it had a significantly higher score than the other models. 

In [32]:
stackCVreg = StackingCVRegressor(regressors=(xgb_model,lr_model,br_model,lightgbm_model,svr_model,
                                            rfr_model,gbr_model,ridge_model),
                                meta_regressor=lr_model,
                                use_features_in_secondary=True)

In [33]:
print(rmse_cv(stackCVreg,np.array(X),np.array(y)).mean())

[19:35:45] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:35:46] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:35:48] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:35:50] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:35:52] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

Ill-conditioned matrix (rcond=3.55632e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.54244e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.54515e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.69711e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.66968e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.59768e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.99578e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.80743e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.84225e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.53439e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.8079e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.47533e-17): result may not be accurate.
Ill-conditioned matrix (rcond=4.23814e-22): result may not be accurate.
Ill-conditioned matrix (rcond=3.42115e-17): result may not be acc

[19:37:33] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


Ill-conditioned matrix (rcond=2.74251e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.75574e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.83786e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78503e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.72177e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.92576e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.70233e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.67385e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78952e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.88052e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.79337e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.82863e-17): result may not be accurate.


[19:37:59] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:38:01] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:38:03] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:38:05] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[19:38:06] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

...

[20:02:52] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:02:55] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:02:57] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:03:00] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:03:03] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

Ill-conditioned matrix (rcond=4.05144e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.69728e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.73646e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.95019e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.63509e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.7965e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.74447e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.77064e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.77926e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.86224e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.92364e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.85222e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.17503e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.32345e-17): result may not be acc

[20:05:03] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


Ill-conditioned matrix (rcond=2.63092e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.86984e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78341e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.74794e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.81388e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.86971e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.76513e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.64043e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78806e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.6835e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.72277e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.9172e-17): result may not be accurate.


16145.037637199292


To fit all the models in preparation creating an overall blended model. 

In [34]:
cat_model_all_data = cat_model.fit(X,y)
xgb_model_all_data = xgb_model.fit(X,y)
lr_model_all_data = lr_model.fit(X,y)
br_model_all_data = br_model.fit(X,y)
lightgbm_model_all_data = lightgbm_model.fit(X,y)
svr_model_all_data = svr_model.fit(X,y)
rfr_model_all_data = rfr_model.fit(X,y)
gbr_model_all_data = gbr_model.fit(X,y)
ridge_model_all_data = ridge_model.fit(X,y)

0:	learn: 0.3814251	total: 1.93ms	remaining: 1.93s
1:	learn: 0.3645525	total: 4.24ms	remaining: 2.11s
2:	learn: 0.3497168	total: 5.8ms	remaining: 1.93s
3:	learn: 0.3351457	total: 7.32ms	remaining: 1.82s
4:	learn: 0.3242337	total: 9.24ms	remaining: 1.84s
5:	learn: 0.3113050	total: 11.1ms	remaining: 1.84s
6:	learn: 0.3018370	total: 13.1ms	remaining: 1.86s
7:	learn: 0.2915334	total: 14.8ms	remaining: 1.84s
8:	learn: 0.2810088	total: 16.9ms	remaining: 1.86s
9:	learn: 0.2723325	total: 18.7ms	remaining: 1.85s
10:	learn: 0.2640630	total: 20.4ms	remaining: 1.84s
11:	learn: 0.2558824	total: 22.2ms	remaining: 1.83s
12:	learn: 0.2484973	total: 24.9ms	remaining: 1.89s
13:	learn: 0.2411889	total: 26.9ms	remaining: 1.89s
14:	learn: 0.2344371	total: 29.3ms	remaining: 1.93s
15:	learn: 0.2283734	total: 31.7ms	remaining: 1.95s
16:	learn: 0.2230633	total: 33.7ms	remaining: 1.95s
17:	learn: 0.2178149	total: 35.6ms	remaining: 1.94s
18:	learn: 0.2130425	total: 37.3ms	remaining: 1.92s
19:	learn: 0.2093456	to

177:	learn: 0.1100048	total: 342ms	remaining: 1.58s
178:	learn: 0.1099563	total: 344ms	remaining: 1.57s
179:	learn: 0.1097832	total: 346ms	remaining: 1.58s
180:	learn: 0.1097091	total: 348ms	remaining: 1.57s
181:	learn: 0.1095791	total: 349ms	remaining: 1.57s
182:	learn: 0.1094588	total: 351ms	remaining: 1.57s
183:	learn: 0.1093392	total: 353ms	remaining: 1.57s
184:	learn: 0.1092540	total: 355ms	remaining: 1.56s
185:	learn: 0.1090133	total: 357ms	remaining: 1.56s
186:	learn: 0.1090012	total: 359ms	remaining: 1.56s
187:	learn: 0.1089463	total: 361ms	remaining: 1.56s
188:	learn: 0.1087722	total: 362ms	remaining: 1.55s
189:	learn: 0.1086178	total: 364ms	remaining: 1.55s
190:	learn: 0.1083676	total: 366ms	remaining: 1.55s
191:	learn: 0.1082657	total: 368ms	remaining: 1.55s
192:	learn: 0.1081645	total: 369ms	remaining: 1.54s
193:	learn: 0.1081106	total: 371ms	remaining: 1.54s
194:	learn: 0.1080453	total: 373ms	remaining: 1.54s
195:	learn: 0.1079861	total: 378ms	remaining: 1.55s
196:	learn: 

337:	learn: 0.0943748	total: 718ms	remaining: 1.41s
338:	learn: 0.0943007	total: 724ms	remaining: 1.41s
339:	learn: 0.0941440	total: 728ms	remaining: 1.41s
340:	learn: 0.0940837	total: 731ms	remaining: 1.41s
341:	learn: 0.0940792	total: 735ms	remaining: 1.41s
342:	learn: 0.0940384	total: 738ms	remaining: 1.41s
343:	learn: 0.0939544	total: 741ms	remaining: 1.41s
344:	learn: 0.0939325	total: 745ms	remaining: 1.41s
345:	learn: 0.0938274	total: 748ms	remaining: 1.41s
346:	learn: 0.0936654	total: 752ms	remaining: 1.41s
347:	learn: 0.0936465	total: 759ms	remaining: 1.42s
348:	learn: 0.0936282	total: 761ms	remaining: 1.42s
349:	learn: 0.0936247	total: 764ms	remaining: 1.42s
350:	learn: 0.0936046	total: 768ms	remaining: 1.42s
351:	learn: 0.0934034	total: 771ms	remaining: 1.42s
352:	learn: 0.0932520	total: 774ms	remaining: 1.42s
353:	learn: 0.0931643	total: 776ms	remaining: 1.42s
354:	learn: 0.0931274	total: 779ms	remaining: 1.41s
355:	learn: 0.0930235	total: 781ms	remaining: 1.41s
356:	learn: 

581:	learn: 0.0797471	total: 1.28s	remaining: 921ms
582:	learn: 0.0796459	total: 1.28s	remaining: 920ms
583:	learn: 0.0795506	total: 1.29s	remaining: 917ms
584:	learn: 0.0795471	total: 1.29s	remaining: 915ms
585:	learn: 0.0794852	total: 1.29s	remaining: 912ms
586:	learn: 0.0794672	total: 1.29s	remaining: 910ms
587:	learn: 0.0794596	total: 1.29s	remaining: 907ms
588:	learn: 0.0794504	total: 1.3s	remaining: 905ms
589:	learn: 0.0793423	total: 1.3s	remaining: 903ms
590:	learn: 0.0792836	total: 1.3s	remaining: 900ms
591:	learn: 0.0792613	total: 1.3s	remaining: 898ms
592:	learn: 0.0792042	total: 1.3s	remaining: 895ms
593:	learn: 0.0791401	total: 1.31s	remaining: 893ms
594:	learn: 0.0791285	total: 1.31s	remaining: 890ms
595:	learn: 0.0791238	total: 1.31s	remaining: 888ms
596:	learn: 0.0790606	total: 1.31s	remaining: 885ms
597:	learn: 0.0790109	total: 1.31s	remaining: 883ms
598:	learn: 0.0789931	total: 1.31s	remaining: 881ms
599:	learn: 0.0789569	total: 1.32s	remaining: 878ms
600:	learn: 0.078

762:	learn: 0.0723150	total: 1.66s	remaining: 515ms
763:	learn: 0.0722605	total: 1.66s	remaining: 513ms
764:	learn: 0.0722420	total: 1.66s	remaining: 511ms
765:	learn: 0.0721992	total: 1.67s	remaining: 509ms
766:	learn: 0.0721568	total: 1.67s	remaining: 507ms
767:	learn: 0.0721052	total: 1.67s	remaining: 504ms
768:	learn: 0.0720449	total: 1.67s	remaining: 502ms
769:	learn: 0.0720274	total: 1.67s	remaining: 500ms
770:	learn: 0.0720074	total: 1.68s	remaining: 498ms
771:	learn: 0.0719532	total: 1.68s	remaining: 496ms
772:	learn: 0.0719419	total: 1.68s	remaining: 494ms
773:	learn: 0.0719365	total: 1.68s	remaining: 492ms
774:	learn: 0.0719098	total: 1.69s	remaining: 490ms
775:	learn: 0.0718918	total: 1.69s	remaining: 487ms
776:	learn: 0.0718750	total: 1.69s	remaining: 485ms
777:	learn: 0.0718193	total: 1.69s	remaining: 483ms
778:	learn: 0.0718169	total: 1.69s	remaining: 481ms
779:	learn: 0.0717320	total: 1.7s	remaining: 479ms
780:	learn: 0.0716731	total: 1.72s	remaining: 481ms
781:	learn: 0

934:	learn: 0.0665948	total: 2.03s	remaining: 141ms
935:	learn: 0.0665388	total: 2.03s	remaining: 139ms
936:	learn: 0.0665057	total: 2.04s	remaining: 137ms
937:	learn: 0.0664823	total: 2.04s	remaining: 135ms
938:	learn: 0.0664809	total: 2.04s	remaining: 133ms
939:	learn: 0.0664661	total: 2.04s	remaining: 130ms
940:	learn: 0.0664643	total: 2.04s	remaining: 128ms
941:	learn: 0.0664398	total: 2.04s	remaining: 126ms
942:	learn: 0.0664003	total: 2.05s	remaining: 124ms
943:	learn: 0.0663858	total: 2.05s	remaining: 122ms
944:	learn: 0.0663593	total: 2.05s	remaining: 119ms
945:	learn: 0.0662655	total: 2.05s	remaining: 117ms
946:	learn: 0.0662553	total: 2.05s	remaining: 115ms
947:	learn: 0.0662414	total: 2.05s	remaining: 113ms
948:	learn: 0.0662322	total: 2.06s	remaining: 111ms
949:	learn: 0.0661894	total: 2.06s	remaining: 108ms
950:	learn: 0.0661872	total: 2.06s	remaining: 106ms
951:	learn: 0.0660951	total: 2.06s	remaining: 104ms
952:	learn: 0.0660692	total: 2.06s	remaining: 102ms
953:	learn: 

Ill-conditioned matrix (rcond=2.54807e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.58394e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.43973e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.50116e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.54209e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.61996e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.60428e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.77327e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.55501e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.42849e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.53396e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.55496e-17): result may not be accurate.


In [35]:
stackCVreg_model = stackCVreg.fit(np.array(X), np.array(y))

[20:06:15] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:06:18] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:06:20] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:06:23] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:06:26] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

Ill-conditioned matrix (rcond=5.17763e-21): result may not be accurate.
Ill-conditioned matrix (rcond=3.32938e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.35094e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.27908e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.29162e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.41221e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.13627e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.23661e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.23412e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.32594e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.38932e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.34265e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.24077e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.05556e-17): result may not be ac

[20:08:26] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


Ill-conditioned matrix (rcond=2.54807e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.58394e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.43973e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.50116e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.54209e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.61996e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.60428e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.77327e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.55501e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.42849e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.53396e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.55496e-17): result may not be accurate.


Many of the Kaggle submissions used this type of blending approach. Using 35% for the StackingCVRegressor produced the lowest Kaggle score. Both 30% and 40% produced higher scores. 

In [36]:
# Blend models in order to make the final predictions more robust to overfitting
def blended_predictions(X):
    return ((.1 * ridge_model_all_data.predict(X)) + \
            (.1 * svr_model_all_data.predict(X)) + \
            (.1 * cat_model_all_data.predict(X)) + \
            (.1 * gbr_model_all_data.predict(X)) + \
            (.1 * xgb_model_all_data.predict(X)) + \
            (.1 * lightgbm_model_all_data.predict(X)) + \
            (.05 * rfr_model_all_data.predict(X)) + \
            (0.35 * stackCVreg_model.predict(np.array(X))))

In [37]:
def rmsle(y, y_pred):
    random_state=42
    return np.sqrt(mean_squared_error(y, y_pred))

In [38]:
# Get final precitions from the blended model
blended_score = rmsle(y, blended_predictions(X))
print('RMLSE score on train data:')
print(blended_score)

RMLSE score on train data:
0.08534343999399366


In [39]:
submit = test[['Id']]
submit = submit.reset_index(drop=True)

In [40]:
submit_predict = blended_predictions(test)
submit_predict = np.exp(submit_predict)

In [41]:
submit_predict = np.expm1(blended_predictions(test))

In [42]:
submit['SalePrice'] = submit_predict

In [43]:
submit.to_csv('submit_blended_best_data.csv', index=False)